In [1]:
import requests
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(wiki_url)

In [3]:
soup = BeautifulSoup(resp.text)

In [4]:
nbhood_data = []
print(soup.select('.wikitable')[0].find_all('tr')[0].getText())
for i in soup.select('.wikitable')[0].find_all('tr')[1:]:
    pcode, bor, nbhood = [y.strip() for y in [x.getText() for x in i.find_all('td')]]
    # print(pcode, bor, nbhood)
    if bor != 'Not assigned':
        if nbhood == 'Not assigned':
            nbhood = bor
        nbhood_data.append((pcode, bor, nbhood))
t = [i[0] for i in nbhood_data]
print(len(t) == len(set(t))) # true - No entries in the current table have duplicate postal codes.


Postal Code

Borough

Neighbourhood

[('M3A', 'North York', 'Parkwoods'), ('M4A', 'North York', 'Victoria Village'), ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'), ('M6A', 'North York', 'Lawrence Manor, Lawrence Heights'), ('M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"), ('M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'), ('M1B', 'Scarborough', 'Malvern, Rouge'), ('M3B', 'North York', 'Don Mills'), ('M4B', 'East York', 'Parkview Hill, Woodbine Gardens'), ('M5B', 'Downtown Toronto', 'Garden District, Ryerson'), ('M6B', 'North York', 'Glencairn'), ('M9B', 'Etobicoke', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'), ('M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'), ('M3C', 'North York', 'Don Mills'), ('M4C', 'East York', 'Woodbine Heights'), ('M5C', 'Downtown Toronto', 'St. James Town'), ('M6C', 'York', 'Humewood-Cedarvale'), ('M9C', 'Etobicoke', 'Eringate, Bloordale Gardens, Old Burnhamthorpe, 

In [12]:
# define the dataframe columns
column_names = ['Postal Code', 'Borough', 'Neighborhood'] #, 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for postalCode, bor, nbh in nbhood_data:
    neighborhoods = neighborhoods.append({'Postal Code': postalCode, 'Borough': bor, 'Neighborhood': nbh}, ignore_index=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(neighborhoods)

    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
2